In [1]:
from IPython.display import display, Markdown
import joblib
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import ShuffleSplit, GridSearchCV, KFold, cross_validate
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
df = pd.read_csv('../data/raw/wines_SPA.csv')
df_dict = pd.read_csv('../data/external/dict.csv')

In [3]:
df_dict

,variavel,descricao,tipo,subtipo
0,winery,Vinícola: local onde o vinho foi produzido,qualitativa,nominal
1,wine,vinho,qualitativa,nominal
2,year,ano,quantitativa,discreta
3,rating,avaliação: de 4.2 a 4.9,quantitativa,discreta
4,num_reviews,número de avaliações,quantitativa,discreta
5,country,país de origem,qualitativa,nominal
6,region,região do vinho,qualitativa,nominal
7,price,preço em euros,quantitativa,continua
8,type,tipo do vinho,qualitativa,nominal
9,body,"valor do ""corpo"" do vinho: de 2 a 5",quantitativa,discreta


In [4]:
df['year'] = df['year'].replace('N.V.', -1)
df['year'] = df['year'].replace(np.nan, -1)
df['year'] = df['year'].astype(int)
df['year'] = df['year'].replace(-1, np.nan)

In [5]:
df['year']

0       2013.0
1       2018.0
2       2009.0
3       1999.0
4       1996.0
         ...  
7495    2016.0
7496    2018.0
7497    2017.0
7498    2011.0
7499    2016.0
Name: year, Length: 7500, dtype: float64

In [6]:
target_column = 'wine'
nominal_columns = (
    df_dict
    .query("subtipo == 'nominal' and variavel != @target_column")
    .variavel
    .to_list()
)
continuous_columns = (
    df_dict
    .query("subtipo == 'continua'")
    .variavel
    .to_list()
)
discrete_columns = (
    df_dict
    .query("subtipo == 'discreta'")
    .variavel
    .to_list()
)

X = df.drop(columns=[target_column], axis=1)
y = df[target_column]

In [7]:
nominal_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='most_frequent')), # tratamento de dados faltantes
    ('encoding', OneHotEncoder(sparse_output=False, drop='first')), # codificação de variáveis
    ('normalization', StandardScaler()) # normalização de dados
])
continuous_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='mean')), # tratamento de dados faltantes
    ('normalization', StandardScaler()) # normalização de dados
])
discrete_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='median')), # tratamento de dados faltantes
    ('normalization', StandardScaler()) # normalização de dados
])
preprocessor = ColumnTransformer([
    ('nominal', nominal_preprocessor, nominal_columns),
    ('continuous', continuous_preprocessor, continuous_columns),
    ('discrete', discrete_preprocessor, discrete_columns)
])

model = LogisticRegression()